In [61]:
import pandas as pd
import numpy as np

# 데이터 불러오기
train = pd.read_csv('X_train.csv', encoding = 'cp949')
test = pd.read_csv('X_test.csv', encoding = 'cp949')
target = pd.read_csv('y_train.csv', encoding = 'cp949')

# 수치형, 범주형 변수 분류
numerical_feats = train.dtypes[train.dtypes != 'object'].index
categorical_feats = train.dtypes[train.dtypes == 'object'].index

# 결측치는 0으로 확인 (환불 없음)
train.fillna(0, inplace = True)
test.fillna(0, inplace = True)

# 이상치 제거
def drop_outliers(data, n, features):
    outlier_indices = []
    for col in features:
        Q1 = np.percentile(data[col], 25)
        Q3 = np.percentile(data[col], 75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR
        outlier_list_col = data[(data[col] < Q1 - outlier_step) | (data[col] > Q3 + outlier_step)].index
        outlier_indices.extend(outlier_list_col)
    count_dic = {}
    for element in outlier_indices:
        if element not in count_dic:
            count_dic[element] = 0
        count_dic[element] += 1
    multiple_outliers = list(k for k, v in count_dic.items() if v > n)
    return multiple_outliers

outlier_to_drop = drop_outliers(train, 2, numerical_feats)
train.drop(outlier_to_drop, axis = 0, inplace = True)
target.drop(outlier_to_drop, axis = 0, inplace = True)
train.reset_index(drop = True, inplace = True)
target.reset_index(drop = True, inplace = True)

# 왜도 첨도 정규화
for i in numerical_feats:
    print(i, '{:.2f}'.format(train[i].skew()), '{:.2f}'.format(train[i].kurt()))
    
train['총구매액'] = train['총구매액'].apply(lambda x : 0 if x < 0 else x)
test['총구매액'] = test['총구매액'].apply(lambda x : 0 if x < 0 else x)

train['최대구매액'] = train['최대구매액'].apply(lambda x : 0 if x < 0 else x)
test['최대구매액'] = test['최대구매액'].apply(lambda x : 0 if x < 0 else x)

train['총구매액'] = np.log1p(train['총구매액']) 
train['최대구매액'] = np.log1p(train['최대구매액']) 
train['환불금액'] = np.log1p(train['환불금액']) 
train['내점일수'] = np.log1p(train['내점일수']) 
train['내점당구매건수'] = np.log1p(train['내점당구매건수']) 

test['총구매액'] = np.log1p(test['총구매액']) 
test['최대구매액'] = np.log1p(test['최대구매액']) 
test['환불금액'] = np.log1p(test['환불금액']) 
test['내점일수'] = np.log1p(test['내점일수']) 
test['내점당구매건수'] = np.log1p(test['내점당구매건수']) 

# FE
    
train['환불여부'] = train['환불금액'].apply(lambda x:'refund' if x > 0 else 'not_refund')
test['환불여부'] = test['환불금액'].apply(lambda x:'refund' if x > 0 else 'not_refund')

train['내점당구매액'] = train['총구매액']/(train['내점일수']*train['내점당구매건수'])
test['내점당구매액'] = test['총구매액']/(test['내점일수']*test['내점당구매건수'])

train = pd.get_dummies(train, columns = ['주구매지점', '주구매상품', '환불여부'], prefix = ['주구매지점', '주구매상품', '환불여부'])
test = pd.get_dummies(test, columns = ['주구매지점', '주구매상품', '환불여부'], prefix = ['주구매지점', '주구매상품', '환불여부'])

for i in train.columns:
    if i not in test.columns:
        print(i)
        train.drop([i], axis = 1, inplace = True)

# 데이터 분할 및 모델링 준비
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score

target = target['gender']
test_cust = test['cust_id']
train.drop(['cust_id'], axis = 1, inplace = True)
test.drop(['cust_id'], axis = 1, inplace = True)

# 모델링
skf = StratifiedKFold(n_splits = 5, random_state = 2021, shuffle = True)

RF_models={}

for n_fold, (train_index, val_index) in enumerate(skf.split(train, target)):
        trn_x, val_x = train.iloc[train_index], train.iloc[val_index]
        trn_y, val_y = target.iloc[train_index], target.iloc[val_index]
        model = RandomForestClassifier(1000, random_state=2021)
        model.fit(trn_x, trn_y)
        pred = model.predict_proba(val_x)
        print(roc_auc_score(val_y, pred[:,1:]))
        RF_models[n_fold] = model


pred_list = []
for fold in range(5):
    pred = RF_models[fold].predict_proba(test)[:, 1]
    pred_list.append(pred)

preds = np.asarray(pred_list).mean(axis=0)
# 변수 중요도 확인

for i in range(len(train.columns)):
    print('{}: {:.2f}'.format(train.columns[i], model.feature_importances_[i]*100))

# 제출파일 생성
sub = pd.DataFrame()
sub["cust_id"] = test_cust
sub["gender"] = preds
sub.to_csv("0000.csv", index=False)

cust_id 0.00 -1.20
총구매액 3.08 14.30
최대구매액 3.95 26.00
환불금액 14.57 326.82
내점일수 2.96 15.07
내점당구매건수 3.05 16.46
주말방문비율 0.92 0.00
구매주기 2.01 5.13
주구매상품_소형가전
0.6404361892882201
0.6489338894519373
0.6641917899220413
0.6409947050663332
0.6644067134288115
총구매액: 11.77
최대구매액: 12.16
환불금액: 3.53
내점일수: 6.83
내점당구매건수: 8.25
주말방문비율: 7.68
구매주기: 7.50
내점당구매액: 11.77
주구매지점_강남점: 0.66
주구매지점_관악점: 0.25
주구매지점_광주점: 0.68
주구매지점_노원점: 0.55
주구매지점_대구점: 0.04
주구매지점_대전점: 0.41
주구매지점_동래점: 0.30
주구매지점_미아점: 0.55
주구매지점_본  점: 1.75
주구매지점_부산본점: 0.92
주구매지점_부평점: 0.30
주구매지점_분당점: 1.16
주구매지점_상인점: 0.00
주구매지점_센텀시티점: 0.04
주구매지점_안양점: 0.22
주구매지점_영등포점: 1.00
주구매지점_울산점: 0.08
주구매지점_인천점: 0.30
주구매지점_일산점: 0.84
주구매지점_잠실점: 1.30
주구매지점_전주점: 0.12
주구매지점_창원점: 0.03
주구매지점_청량리점: 0.51
주구매지점_포항점: 0.11
주구매상품_가공식품: 1.17
주구매상품_가구: 0.09
주구매상품_건강식품: 0.32
주구매상품_골프: 0.58
주구매상품_구두: 0.36
주구매상품_기타: 1.20
주구매상품_남성 캐주얼: 0.60
주구매상품_남성 트랜디: 0.02
주구매상품_남성정장: 0.25
주구매상품_농산물: 0.89
주구매상품_대형가전: 0.06
주구매상품_디자이너: 0.91
주구매상품_란제리/내의: 0.09
주구매상품_명품: 0.45
주구매상품_모피/피혁: 0.41
주구매상품_보석: 0.04
주구